# 导入

In [1]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 1.5 MB/s eta 0:00:00


In [2]:
!pip install geopandas

In [3]:
!pip install dgl==0.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 12.6 MB/s eta 0:00:00


In [4]:
!pip install utm

  Preparing metadata (setup.py) ... done
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=e4e5c6d1e0e1d8c089e00ea2b79d7e5ddaa0df56973bf2db0653aa27248f5cea
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
import pandas as pd
import  pandas  as pd
from  pandas import  DataFrame as df
import numpy as np
# import skfuzzy as fuzz
# from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import math
import os
import sys
import shapely
from shapely.geometry import *
import dgl
import osmnx as ox
import geopandas as gpd
import xml.etree.ElementTree as ET
from functools import lru_cache
from pathlib import Path
from typing import Any, List, Sequence, Union
import torch
import torch.utils.data as torch_data

import utm

import torch.nn as nn
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader
from torch.autograd import Variable

import datetime

/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/requests/__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.2.0) doesn't match a supported version!
  RequestsDependencyWarning,


# config

In [2]:
max_nodes = 20
max_lanes = 8
max_frames = 50
max_objects = 10
num_agents = 4
#特殊值处理
MAX_V = 100

In [12]:
class DefaultConfig(object):
    env = 'default' # visdom 环境
    model = 'MTR' # 使用的模型，名字必须与models/__init__.py中的名字一致

    train_data_root = '/content/gdrive/MyDrive/exitD/data/train_mtr' # 训练集存放路径
    # test_data_root = './data/test' # 测试集存放路径
    load_model_path = None # 加载预训练的模型的路径，为None代表不加载

    batch_size = 20 # batch size
    use_gpu = False # use GPU or not
    num_workers = 4 # how many workers for loading data
    print_freq = 20 # print info every N batch

    # agent
    num_agents = 4

    # map feature
    num_points_polyline = 10
    num_src_polyline = 20

    result_file = 'result.csv'

    max_epoch = 100
    lr = 0.01 # initial learning rate
    lr_decay = 0.97 # when val_loss increase, lr = lr*lr_decay
    weight_decay = 1e-4 # 损失函数

    def parse(self, kwargs):
        '''
        根据字典kwargs 更新 config参数
        '''
        # 更新配置参数
        for k, v in kwargs.items():
            if not hasattr(self, k):
                # 警告还是报错，取决于你个人的喜好
                warnings.warn("Warning: opt has not attribut %s" %k)
            setattr(self, k, v)

        # 打印配置信息
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('__'):
                print(k, getattr(self, k))

In [13]:
opt = DefaultConfig()

# DataSet

## train data

In [ ]:
def read_osm(file_name):
  '''解析osm并导出csv
  '''
  file_path = '/content/gdrive/MyDrive/exitD/lanelet2/' + file_name
  G = ox.graph_from_xml(file_path, simplify=False, retain_all=True)
  gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
  gdf_nodes.to_csv('/content/gdrive/MyDrive/exitD/data/gdf_nodes_' + file_name.split('.')[0] + '.csv',encoding='utf-8')
  gdf_edges.to_csv('/content/gdrive/MyDrive/exitD/data/gdf_edges_' + file_name.split('.')[0] + '.csv', encoding='utf-8')

  return gdf_nodes, gdf_edges

In [ ]:
# file_id = '00'
# tracks_path = '/content/gdrive/MyDrive/exitD/data/' + file_id + '_tracks.csv'
# trackmeta_path = '/content/gdrive/MyDrive/exitD/data/' + file_id + '_tracksMeta.csv'
# total_data = pd.read_csv(tracks_path)
# data=df(total_data[['frame', 'trackId', 'xCenter', 'yCenter', 'lonVelocity', 'latVelocity',\
#             'width', 'length', 'heading','leadId', 'rearId', \
#                   'leftLeadId', 'rightLeadId', 'leftRearId', 'rightRearId']])
# del total_data
# type_data = pd.read_csv(trackmeta_path)
# type_data = type_data[['recordingId', 'trackId', 'class']]
# type_data = type_data[type_data['recordingId']==int(file_id[0])]
# data = pd.merge(data, type_data, on='trackId', how='left')
# data.head()



<ipython-input-5-24549b83e78f>:4: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv(tracks_path)


,frame,trackId,xCenter,yCenter,lonVelocity,latVelocity,width,length,heading,leadId,rearId,leftLeadId,rightLeadId,leftRearId,rightRearId,recordingId,class
0,0,0,380.7679,-200.3428,31.9832,-0.0,2.0046,4.884,127.4497,-1,3,-1,-1,-1,5,0,car
1,1,0,379.9906,-199.3281,32.0047,-0.0,2.0046,4.884,127.4427,-1,3,-1,-1,-1,5,0,car
2,2,0,379.2125,-198.3121,32.0264,-0.0,2.0046,4.884,127.4357,-1,3,-1,-1,-1,5,0,car
3,3,0,378.4335,-197.2947,32.0478,-0.0,2.0046,4.884,127.4285,-1,3,-1,-1,-1,5,0,car
4,4,0,377.6538,-196.2763,32.0687,-0.0,2.0046,4.884,127.4211,-1,3,-1,-1,-1,5,0,car


In [ ]:
df['class'][0]

'car'

In [9]:
def get_traj(file_id, nums, opt):
  '''解析csv并导出轨迹片段 并解析车-车道对应
  args:
    file_id: recording
    nums: 截取片段数量
  return:
    agent_lane_dict, key: track_id, value: lane_id
  '''
  tracks_path = '/content/gdrive/MyDrive/exitD/data/' + file_id + '_tracks.csv'
  trackmeta_path = '/content/gdrive/MyDrive/exitD/data/' + file_id + '_tracksMeta.csv'
  total_data = pd.read_csv(tracks_path)
  # data=pd.DataFrame(total_data[['frame', 'trackId', 'laneletId', 'xCenter', 'yCenter', 'lonVelocity', 'latVelocity',\
  #             'width', 'length', 'heading','leadId', 'rearId', \
  #                   'leftLeadId', 'rightLeadId', 'leftRearId', 'rightRearId']])
  data=pd.DataFrame(total_data[['frame', 'trackId', 'laneletId', 'xCenter', 'yCenter', 'xVelocity', 'yVelocity',\
              'width', 'length', 'heading','leadId', 'rearId', \
                    'leftLeadId', 'rightLeadId', 'leftRearId', 'rightRearId']])
  del total_data
  type_data = pd.read_csv(trackmeta_path)
  type_data = type_data[['recordingId', 'trackId', 'class']]
  type_data = type_data[type_data['recordingId']==int(file_id)]
  data = pd.merge(data, type_data, on='trackId', how='left')

  agent_lane_dict = dict()
  for id in list(data['trackId'].unique()):
    lanes = list(data[data['trackId']==1]['laneletId'].unique())
    agent_lane_dict[id] = list()
    for lane in lanes:
      if len(lane) > 5:
        agent_lane_dict[id].append(lane.split(';')[0])
        agent_lane_dict[id].append(lane.split(';')[1])
      else:
        agent_lane_dict[id].append(lane)

  print(agent_lane_dict[1])

  # print(data.head(3))

  i = 1
  row = 0
  while i < nums+1:
    #每50帧截取一个csv
    df = data[(data['frame'] >= row) & (data['frame'] < row+50)].reset_index()
    ids_50 = list(set(list(df[df['frame'] == row]['trackId'].unique()))\
                  & set(list(df[df['frame'] == (row+49)]['trackId'].unique())))
    print(ids_50)  #50帧内一直存在的车辆
    if len(ids_50) > opt.num_agents:
      mid = int(len(ids_50) / 2)
      # print(mid-(opt.num_agents/2), mid+(opt.num_agents/2))
      agent_id = set(ids_50[int(mid-(opt.num_agents/2)):int(mid+(opt.num_agents/2))])  #选预测目标
      print('agent: ', agent_id)

      for j in range(len(df)):
        if df['trackId'][j] in agent_id:
          df['class'][j] = 'agent'

      # print(track_df.head(2))
      df.sort_values(by=['frame', 'trackId'],ascending=True)
      df.to_csv('/content/gdrive/MyDrive/exitD/data/train_mtr/' + str(i) + '.csv', index=False)
      i += 1

    row += 100

  return agent_lane_dict

test

In [14]:
# get_traj('00', 200, opt)
agent_lane_dict = get_traj('00', 2, opt)

<ipython-input-9-b7ec8021abc7>:11: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv(tracks_path)


['1707', '1710', '1959']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
agent:  {8, 5, 6, 7}


<ipython-input-9-b7ec8021abc7>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'][j] = 'agent'


[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
agent:  {16, 13, 14, 15}


<ipython-input-9-b7ec8021abc7>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'][j] = 'agent'


## lane_dict

In [ ]:
# import utm

# utm_ = utm.from_latlon(latitude=50.99182381446, longitude=6.89598424975)
# print(utm_[0], utm_[1])

352342.97600469075 5651022.793593203


In [15]:
def get_lane_dict(file_name):
  file_path = '/content/gdrive/MyDrive/exitD/lanelet2/' + file_name
  G = ox.graph_from_xml(file_path, simplify=False, retain_all=True)
  gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

  # 车道对应的中心线向量
  lane_centerlines_dict = dict()  #key(lane_id), value(lane_props)

  i = 0
  id_list = []
  for id in gdf_edges['osmid']:
    id_list.append(id)
    if id not in lane_centerlines_dict:
      lane_centerlines_dict[id] = []
    i += 1

  i = 0
  reverse_list = []
  for r in gdf_edges['reversed']:
    reverse_list.append(r)
    i += 1

  i = 0
  for line in gdf_edges['geometry']:
    line = str(line)
    # print(str(line).split(' (')[2])
    x0 = float(line.split('(')[1].split(' ')[0])
    y0 = float(line.split(' ')[2].split(',')[0])
    # long, lat -> UTM
    xy = utm.from_latlon(latitude=y0, longitude=x0)
    x0 = xy[0] - 352146.6
    y0 = xy[1] - 5651141.9

    id = id_list[i]
    reverse = reverse_list[i]
    if not reverse:
      lane_centerlines_dict[id].append([x0, y0])
    i += 1

  print(lane_centerlines_dict[1500])

  # 车道对应左右车道线
  tree = ET.parse(file_path)
  # 获取根节点
  root = tree.getroot()
  ref_df = []  #id, left, right

  for r in root.iter('relation'):
    # print(r.attrib)
    relation = [r.attrib['id'], "", ""]
    for mem in r[0:2]:
      # print(mem.attrib)
      # print(len(mem.attrib))
      if mem.attrib['role'] == 'left':
        relation[1] = mem.attrib['ref']
      elif mem.attrib['role'] == 'right':
        relation[2] = mem.attrib['ref']
    ref_df.append(relation)

  #建立 lanlet_id 与左右车道映射
  left_id_dict = dict()
  right_id_dict = dict()
  for i in range(len(ref_df)):
    left_id_dict[ref_df[i][0]] = ref_df[i][1]
    right_id_dict[ref_df[i][0]] = ref_df[i][2]

  return lane_centerlines_dict, left_id_dict, right_id_dict

test

In [16]:
lane_centerlines_dict, left_id_dict, right_id_dict = get_lane_dict('0_cologne_butzweiler.osm')

[[196.37600469077006, -119.10640679765493], [201.26528693194268, -117.24026796221733], [210.03613715310348, -114.32909260410815], [216.23171656380873, -113.02279563527554], [223.5843022147892, -112.01508004311472], [227.39122416620376, -111.75382153782994], [231.2354697001865, -111.52988456469029], [235.60223327414133, -111.49256178736687], [241.55523357522907, -111.94045365601778], [245.53010797256138, -112.42564969044179], [249.82222625403665, -113.22808900009841], [253.53584178636083, -114.0865129660815], [257.7719760154141, -115.1875343779102], [261.7281897185021, -116.32587840501219], [266.4495193986804, -117.89343486353755], [272.5331307775341, -120.63665882404894], [277.6276883612154, -123.41720422916114], [282.77823043137323, -126.72026920877397], [286.90239615674363, -130.00467308796942], [291.58640353463124, -133.77427285164595], [295.3000191284809, -137.80513133481145]]


In [3]:
import sys
print(sys.path)

['/home/zhangxih21/data/exiD/src', '/home/zhangxih21/miniconda3/envs/py37/lib/python37.zip', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/lib-dynload', '', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages', '/home/zhangxih21/software/OpenCDA', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/seaborn-0.12.2-py3.7.egg', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/scipy-1.5.3-py3.7-linux-x86_64.egg', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/scikit_learn-0.24.2-py3.7-linux-x86_64.egg', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/pygame-2.4.0.dev2-py3.7-linux-x86_64.egg', '/home/zhangxih21/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/extensions', '/home/zhangxih21/.ipython']


In [5]:
pd.read_csv('/home/zhangxih21/data/exiD/data/train/0/2.csvdata/train/0/2.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/zhangxih21/data/exiD/data/train/0/2.csvdata/train/0/2.csv'

In [6]:
#test
sys.path.append(os.path.abspath('/home/zhangxih21/data/exiD/'))
G = ox.graph_from_xml('/data/maps/0_cologne_butzweiler.osm', simplify=False, retain_all=True)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

FileNotFoundError: [Errno 2] No such file or directory: '/data/maps/0_cologne_butzweiler.osm'

## get_lane_centerlines

In [17]:
def get_lane_centerlines(argoverse_data, lane_centerlines_dict):
  '''
  根据车辆位置信息和地图，获取周围的车道信息
  Args:
    argoverse_data: afl, 获取车辆轨迹, 包含车道id, traj_xy
  return:
    lane_centerlines: list [[]]
  '''
  agent_obs_traj = argoverse_data.agent_traj
  track_id = int(argoverse_data.seq_df[argoverse_data.seq_df["class"] == "agent"]["trackId"].unique()[0])
  lane_centerlines_id = agent_lane_dict[track_id]
  # for id in data['laneletId']:
  #   if len(id) < 6:
  #     lane_centerlines_id.add(id)
  #   else:
  #     ids = id.split(';')
  #     lane_centerlines_id.add(id[0])
  #     lane_centerlines_id.add(id[1])

  lane_centerlines = []
  for id in lane_centerlines_id:
    if id in left_id_dict:
      left_id = left_id_dict[id]
      lane_centerlines.append(lane_centerlines_dict[int(left_id)])
    if id in right_id_dict:
      right_id = right_id_dict[id]
      lane_centerlines.append(lane_centerlines_dict[int(right_id)])

  return lane_centerlines


## afl_loader

https://github.com/argoverse/argoverse-api/blob/master/argoverse/data_loading/argoverse_forecasting_loader.py

In [18]:
@lru_cache(128)
def _read_csv(path: Path, *args: Any, **kwargs: Any) -> pd.DataFrame:
    """A caching CSV reader

    Args:
        path: Path to the csv file
        *args, **kwargs: optional arguments to be used while data loading

    Returns:
        pandas DataFrame containing the loaded csv
    """
    return pd.read_csv(path, *args, **kwargs)

In [19]:
class ArgoverseForecastingLoader:
    def __init__(self, root_dir: Union[str, Path]):
        """Initialization function for the class.

        Args:
            root_dir: Path to the folder having sequence csv files
        """
        self.counter: int = 0

        root_dir = Path(root_dir)
        self.seq_list: Sequence[Path] = [(root_dir / x).absolute() for x in os.listdir(root_dir)]

        self.current_seq: Path = self.seq_list[self.counter]

    @property
    def track_id_list(self) -> List[int]:
        """Get the track ids in the current sequence.

        Returns:
            list of track ids in the current sequence
        """
        _track_id_list: List[int] = np.unique(self.seq_df["trackId"].values).tolist()
        return _track_id_list

    @property
    def num_tracks(self) -> int:
        """Get the number of tracks in the current sequence.

        Returns:
            number of tracks in the current sequence
        """
        return len(self.track_id_list)

    @property
    def seq_df(self) -> pd.DataFrame:
        """Get the dataframe for the current sequence.

        Returns:
            pandas DataFrame for the current sequence
        """
        return _read_csv(self.current_seq)

    @property
    def agent_traj(self) -> np.ndarray:
        """Get the trajectory for the track of type 'AGENT' in the current sequence.

        Returns:
            numpy array of shape (seq_len x 2) for the agent trajectory
        """
        #test
        # print(self.seq_df.columns.values)
        agent_id = self.seq_df[self.seq_df["class"] == "agent"]["trackId"].unique()
        agent_x, agent_y = [], []
        for id in list(agent_id):
          agent_x.append(self.seq_df[(self.seq_df["class"] == "agent") \
                        & (self.seq_df["trackId"] == int(id))]["xCenter"])
          agent_y.append(self.seq_df[(self.seq_df["class"] == "agent") \
                        & (self.seq_df["trackId"] == int(id))]["yCenter"])
        # print('agent_x: ', len(agent_x), len(agent_x[0]))
        agent_traj = np.column_stack((agent_x, agent_y))
        # print('agent_traj: ', agent_traj.shape)
        return agent_traj # [4,50,50]

    @property
    def agent_info(self) -> np.ndarray:
        """Get the trajectory for the track of type 'AGENT' in the current sequence.

        Returns:
            numpy array of shape (seq_len x 2) for the agent trajectory
        """
        agent_id = self.seq_df[self.seq_df["class"] == "agent"]["trackId"].unique()
        agent_x, agent_y = [], []
        vel_x, vel_y = [], []
        heading = []
        for id in list(agent_id):
          temp_df = self.seq_df[(self.seq_df["class"] == "agent") \
                        & (self.seq_df["trackId"] == int(id))]
          # print('agent columns ', temp_df.columns.values)
          agent_x.append(temp_df["xCenter"])
          agent_y.append(temp_df["yCenter"])
          vel_x.append(temp_df["xVelocity"])
          vel_y.append(temp_df["yVelocity"])
          heading.append(temp_df["heading"])

        agent_info = np.column_stack((agent_x, agent_y, vel_x, vel_y, heading))
        # print('agent_info: ', agent_info.shape, agent_info[0,:])
        return agent_info # [4,50,5] agents, frames, features

    @property
    def obj_info(self) -> np.ndarray:
        """Get the trajectory for the track of type 'OTHER' in the current sequence.

        Returns:
            numpy array of shape (seq_len x 2) for the agent trajectory
        """
        obj_id = self.seq_df[self.seq_df["class"] != "agent"]["trackId"].unique()
        agent_x, agent_y = [], []
        vel_x, vel_y = [], []
        heading = []
        start_frame = min(self.seq_df["frame"])
        end_frame = max(self.seq_df["frame"])
        # print('start - end ', start_frame, end_frame)
        for id in list(obj_id):
          temp_df = self.seq_df[(self.seq_df["class"] != "agent") \
                        & (self.seq_df["trackId"] == int(id))].reset_index()
          # print(temp_df["xCenter"][0:5])
          # 序列缺失值填补
          df_start = max(0, min(temp_df["frame"]) - start_frame)
          df_end = min(max(temp_df["frame"]) - start_frame + 1, max_frames)
          # print('df ', df_start, df_end, len(temp_df["xCenter"]))
          temp_x, temp_y, temp_vx, temp_vy, temp_h = [0]*max_frames,[0]*max_frames,[0]*max_frames,[0]*max_frames,[0]*max_frames
          # print( temp_df["xCenter"][0])
          for i in range(df_start, df_end):
            temp_x[i] = temp_df["xCenter"][i-df_start]
            temp_y[i] = temp_df["yCenter"][i-df_start]
            temp_vx[i] = temp_df["xVelocity"][i-df_start]
            temp_vy[i] = temp_df["yVelocity"][i-df_start]
            temp_h[i] = temp_df["heading"][i-df_start]
          # print('temp x ', temp_x)
          agent_x.append(temp_x)
          agent_y.append(temp_y)
          vel_x.append(temp_vx)
          vel_y.append(temp_vy)
          heading.append(temp_h)

        obj_info = np.column_stack((agent_x, agent_y, vel_x, vel_y, heading))
        # print('obj_info ', obj_info.shape)
        return obj_info # [4,50,5] objs, frames, features

    def __iter__(self) -> "ArgoverseForecastingLoader":
        """Iterator for enumerating over sequences in the root_dir specified.

        Returns:
            Data Loader object for the first sequence in the data
        """
        self.counter = 0
        return self

    def __next__(self) -> "ArgoverseForecastingLoader":
        """Get the Data Loader object for the next sequence in the data.

        Returns:
            Data Loader object for the next sequence in the data
        """
        if self.counter >= len(self):
            raise StopIteration
        else:
            self.current_seq = self.seq_list[self.counter]
            self.counter += 1
            return self

    def __len__(self) -> int:
        """Get the number of sequences in the data

        Returns:
            Number of sequences in the data
        """
        return len(self.seq_list)

    def __str__(self) -> str:
        """Decorator that returns a string storing some stats of the current sequence

        Returns:
            A string storing some stats of the current sequence
        """
        return f"""Seq : {self.current_seq}
        ----------------------
        || City: {self.city}
        || # Tracks: {len(self.track_id_list)}
        ----------------------"""

    def __getitem__(self, key: int) -> "ArgoverseForecastingLoader":
        """Get the DataLoader object for the sequence corresponding to the given index.

        Args:
            key: index of the element

        Returns:
            Data Loader object for the given index
        """

        self.counter = key
        self.current_seq = self.seq_list[self.counter]
        return self

    def get(self, seq_id: Union[Path, str]) -> "ArgoverseForecastingLoader":
        """Get the DataLoader object for the given sequence path.

        Args:
            seq_id: Fully qualified path to the sequence

        Returns:
            Data Loader object for the given sequence path
        """
        self.current_seq = Path(seq_id).absolute()
        return self

## compose_graph

In [20]:
def compose_graph(lane,label):
    '''
    输入的是单条车道向量
    把车道组织成图,并返回结点特征((x1,y1,vx1,vy1,h1),(x2,y2,vx1,vy1,h1),label)
    Args:
    lane: list, 车道中心线向量
    label: int, 车辆周围车道编号
    '''
    nodeN = lane.shape[0]-1
    lane_dim = len(lane[0])
    # print('nodeN max_nodes ', nodeN, max_nodes)
    features = torch.zeros(max_nodes, 11) #统一变量维度
    graph = dgl.DGLGraph()
    # graph = dgl.graph()
    graph.add_nodes(max_nodes)
    mask = []
    for i in range(max_nodes):
      if i < nodeN:
        for j in range(lane_dim):
          features[i][j] = lane[i][j]
          features[i][j+lane_dim] = lane[i+1][j]

        features[i][2*lane_dim] = label  #torch.tensor(list(lane[i])+list(lane[i+1])+[label])
        mask.append(1)
      else:
        mask.append(0)

    # print('compose_graph ', features.size())
    src = []
    dst = []
    for i in range(nodeN):
      for j in range(nodeN):
        if i != j:
          src.append(i)
          dst.append(j)
    graph.add_edges(src,dst)
    # print('graph ', graph.num_nodes(), features.size())
    graph.ndata['v_feature'] = features
    return graph,features


## compose_agent_graph

In [ ]:
def compose_agent_graph(lane,label):
    '''
    输入的是车道向量
    把车道组织成图,并返回结点特征((x1,y1,vx1,vy1,h1),(x2,y2,vx1,vy1,h1),label)
    Args:
    lane: list, 车道中心线向量
    label: int, 车辆周围车道编号
    '''
    nodeN = lane.shape[0]-1
    #features = torch.zeros(nodeN,5)
    features = torch.zeros(max_nodes,11)
    graph = dgl.DGLGraph()
    #graph.add_nodes(nodeN)
    graph.add_nodes(max_nodes)
    mask = []
    for i in range(max_nodes):
        if i < nodeN:
            features[i][0] = lane[i][0]
            features[i][1] = lane[i][1]
            features[i][2] = lane[i][2]
            features[i][3] = lane[i][3]
            features[i][4] = lane[i][4]

            features[i][5] = lane[i+1][0]
            features[i][6] = lane[i+1][1]
            features[i][7] = lane[i+1][2]
            features[i][8] = lane[i+1][3]
            features[i][9] = lane[i+1][4]

            features[i][10] = label#torch.tensor(list(lane[i])+list(lane[i+1])+[label])
            mask.append(1)
        else:
            mask.append(0)

    src = []
    dst = []
    for i in range(nodeN):
        for j in range(nodeN):
            if i != j:
                src.append(i)
                dst.append(j)
    graph.add_edges(src,dst)
    graph.ndata['v_feature'] = features
    return graph,features

## collate

In [35]:
def collate(samples):
    datas,labels = map(list,zip(*samples))
    AgentGraph = [data['Agent'] for data in datas]
    Center = [data['centerAgent'] for data in datas]
    Mask = [data['Mapmask'] for data in datas]
    # batched_graph = dgl.batch(AgentGraph)

    map_set = []
    feature_set = []
    for index in range(max_lanes):
      map_batch = []
      for data in datas:
        map_batch.append(data['Map'][index])
      mgraph = dgl.batch(map_batch)
      map_set.append(mgraph)
      feature_set.append(mgraph.ndata['v_feature'])

    agent_set = []
    agent_feature_set = []
    for index in range(4): #agents num
      agent_batch = []
      for data in datas:
        agent_batch.append(data['Agent'][index])
      agraph = dgl.batch(agent_batch)
      agent_set.append(agraph)
      agent_feature_set.append(agraph.ndata['v_feature'])

    obj_set = []
    obj_feature_set = []
    for index in range(max_objects): #20
      obj_batch = []
      for data in datas:
        obj_batch.append(data['Obj'][index])
      ograph = dgl.batch(obj_batch)
      obj_set.append(ograph)
      obj_feature_set.append(ograph.ndata['v_feature'])

    new_data = {}
    new_data['Map'] = map_set
    new_data['Mapfeature'] = torch.stack(feature_set,dim = 0)
    # new_data['Agent'] = batched_graph
    # new_data['Agentfeature'] = batched_graph.ndata['v_feature']
    new_data['Agent'] = agent_set
    new_data['Agentfeature'] = torch.stack(agent_feature_set,dim = 0)
    new_data['Obj'] = obj_set
    new_data['Objfeature'] = torch.stack(obj_feature_set,dim = 0)
    new_data['centerAgent'] = Center
    new_data['Mapmask'] = torch.stack(Mask,dim = 0)
    #new_label = []
    #for l in labels:
        #new_label += list(l.flatten())
    # return new_data,torch.stack(labels,dim = 0).reshape(-1,60)
    return new_data,torch.stack(labels,dim = 0).reshape(-1,150) #30*5

## MTRDataset

修改：

- agent: 由 [dx,dy] 新增 [vel_x, vel_y, heading], 改名 center
- obj: 新增非中心的obj, 不预测自己, 辅助预测 agents

TODO:
- 车: 目前改为3维, 需修改对应模型输入维度? 或将 data 拉平展开

注意: 归一化时分母控制范围 + 0.1

In [36]:
class MTRDataset(torch_data.Dataset):
    '''
    数据集
    '''
    def __init__(self, root, train = True,test = False):
        '''
        根据路径获得数据，并根据训练、验证、测试划分数据
        train_data 和 test_data路径分开
        '''
        self.test = test
        self.train = train
        self.afl = ArgoverseForecastingLoader(root)
        # self.avm = ArgoverseMap()
        root_dir = Path(root)
        r = [(root_dir / x).absolute() for x in os.listdir(root_dir)]
        n = len(r)

        if self.test == True:
            self.start = 0
            self.end = n
        elif self.train:
            self.start = 0
            self.end = int(0.7*n)
        else:
            self.start = int(0.7*n)+1
            self.end = n

    def __getitem__(self,index):
        '''
        从csv创建图输入模型
        '''
        data = {}
        # agent_data = pd.read_csv(self.root_dir)
        lane_centerlines = get_lane_centerlines(self.afl[self.start+index], lane_centerlines_dict)
        # print('lane dim ', len(lane_centerlines[0]), len(lane_centerlines[0][0]))
        agent_obs_traj = self.afl[index].agent_traj # num_agents: 4
        center_obs_info = self.afl[index].agent_info #4,250 [x,y,vx,vy,h]
        # print(center_obs_info.shape)
        # print('center_obs_info ', center_obs_info[:,0])
        obj_obs_info = self.afl[index].obj_info
        # print('obj_obs_info ', obj_obs_info.shape)
        data['centerAgent'] = [center_obs_info[1,19], center_obs_info[1,max_frames+19], \
                      center_obs_info[1,2*max_frames+19], center_obs_info[1,3*max_frames+19], \
                        center_obs_info[1,4*max_frames+19]]
        # print(data['centerAgent'])

        #把车道组织成向量和图
        map_set = []
        map_feature = []
        map_mask = []
        #进行norm
        x_min = min(min(center_obs_info[0,0:max_frames]), min(center_obs_info[3,0:max_frames]))
        x_max = max(max(center_obs_info[0,0:max_frames]), max(center_obs_info[3,0:max_frames]))
        y_min = min(min(center_obs_info[0,max_frames:2*max_frames]), min(center_obs_info[3,max_frames:2*max_frames]))
        y_max = max(max(center_obs_info[0,max_frames:2*max_frames]), max(center_obs_info[3,max_frames:2*max_frames]))
        vx_min = min(min(center_obs_info[0,2*max_frames:3*max_frames]), min(center_obs_info[3,2*max_frames:3*max_frames])) #近似
        vx_max = max(max(center_obs_info[0,2*max_frames:3*max_frames]), max(center_obs_info[3,2*max_frames:3*max_frames]))
        vy_min = min(min(center_obs_info[0,3*max_frames:4*max_frames]), min(center_obs_info[3,3*max_frames:4*max_frames]))
        vy_max = max(max(center_obs_info[0,3*max_frames:4*max_frames]), max(center_obs_info[3,3*max_frames:4*max_frames]))
        h_min = min(min(center_obs_info[0,4*max_frames:5*max_frames]), min(center_obs_info[3,4*max_frames:5*max_frames])) #近似
        h_max = max(max(center_obs_info[0,4*max_frames:5*max_frames]), max(center_obs_info[3,4*max_frames:5*max_frames]))
        # print('norm scale ', x_min, x_max, y_min, y_max)

        for lane in lane_centerlines:
          lane = np.array([[(lane[i][0]-data['centerAgent'][0])/(x_max-x_min), \
                    (lane[i][1]-data['centerAgent'][1])/(y_max-y_min)] for i in range(len(lane))]) #归一化
          graph,features = compose_graph(lane,len(map_set))
          map_set.append(graph)
          map_feature.append(features)
          map_mask.append(1)

        # print('map set ', len(map_set))
        if len(map_set) < max_lanes:
          while(len(map_set) < max_lanes):
            #形成空的图，保证大小一致
            lane = np.array([[0,0]])
            graph,features = compose_graph(lane,0)
            map_set.append(graph)
            map_feature.append(features)
            map_mask.append(0)
        else:
          raise Exception("the max lanes is not enough:",len(map_set))

        # print('agent set ', len(center_obs_info))
        center_graphs, center_features, center_traj_norm = [], [], []
        for traj in center_obs_info: #4,250
          # print('center_shape ', traj.shape)
          traj_norm = np.array([[(traj[i]-data['centerAgent'][0])/(x_max-x_min), \
                      (traj[i+max_frames]-data['centerAgent'][1])/(y_max-y_min) ,\
                      (traj[i+2*max_frames]-data['centerAgent'][2])/(vx_max-vx_min+0.1) ,\
                       (traj[i+3*max_frames]-data['centerAgent'][3])/(vy_max-vy_min+0.1) ,\
                      (traj[i+4*max_frames]-data['centerAgent'][4])/(h_max-h_min+0.1)] for i in range(max_frames)]) #归一化
          # print('traj_norm ', traj_norm.shape)
          center_traj_norm.append(traj_norm)
          graph, features = compose_graph(traj_norm[:20],len(center_graphs))
          center_graphs.append(graph)
          center_features.append(features)

        # print('obj set ', len(center_obs_info))
        obj_graphs, obj_features, obj_traj_norm = [], [], []
        #截取固定长度
        if obj_obs_info.shape[0] >= max_objects:
          mid = int(obj_obs_info.shape[0] / 2)
          for i in range(int(mid - max_objects/2), int(mid + max_objects/2)):
            traj = obj_obs_info[i]
            traj_norm = np.array([[(traj[i]-data['centerAgent'][0])/(x_max-x_min), \
                      (traj[i+max_frames]-data['centerAgent'][1])/(y_max-y_min) ,\
                      (traj[i+2*max_frames]-data['centerAgent'][2])/(vx_max-vx_min+0.1) ,\
                      (traj[i+3*max_frames]-data['centerAgent'][3])/(vy_max-vy_min+0.1) ,\
                      (traj[i+4*max_frames]-data['centerAgent'][4])/(h_max-h_min+0.1)] for i in range(max_frames)])
            # traj_norm = (traj - data['centerAgent'])/np.array([x_max-x_min,y_max-y_min,vx_max-vx_min,vy_max-vy_min,h_max-h_min]) #归一化
            obj_traj_norm.append(traj_norm)
            graph, features = compose_graph(traj_norm[:20],len(obj_graphs))
            obj_graphs.append(graph)
            obj_features.append(features)

        else: #填补空车
          while(len(obj_graphs) < max_objects):
            obj = np.array([[0,0,0,0,0]])
            graph,features = compose_graph(obj,0)
            obj_graphs.append(graph)
            obj_features.append(features)

        data['Map'] = map_set # 2dim
        data['Mapfeature'] = map_feature # 2dim
        data['Agent'] = center_graphs  # 2dim 4,250
        data['Agentfeature'] = center_features  # 2dim
        data['Obj'] = obj_graphs  # 2dim 20,250
        data['Objfeature'] = obj_features
        data['Mapmask'] = torch.Tensor(map_mask)
        label = []

        # print(len(center_traj_norm))
        for j in range(len(center_traj_norm)):
          # temp_label = []
          # print(len(center_traj_norm[j]))
          for i in range(19,49): #30*5
            # temp_label.append(torch.Tensor(center_traj_norm[j][i+1]-center_traj_norm[j][i]))
            label.append(torch.Tensor(center_traj_norm[j][i+1]-center_traj_norm[j][i]))
          # label.append(temp_label)

        label = torch.stack(label,dim = 0)
        # print('Agent shape ', len(data['Agent']), data['Agent'][0].num_nodes())
        return data,label.flatten()

    def __len__(self):
        return self.end - self.start


# Model

## MLP

In [22]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(MLP,self).__init__()
        self.MLP = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU()
        )

    def forward(self,x):
        output = self.MLP(x)
        return output

## GCN

$\varphi_{agg}$: 池化, 这里取最大值

In [23]:
import dgl
def gcn_reduce(nodes):
    #agg_feature = torch.max(nodes.mailbox['msg'], dim=1)
    return {'v_feature': torch.max(nodes.mailbox['msg'], dim=1)[0]} #rel agg

In [24]:
class GCNLayer(nn.Module):
    def __init__(self,in_feats,out_feats):
        super(GCNLayer,self).__init__()

    def forward(self,g,inputs):
        gcn_message = dgl.function.copy_src('v_feature','msg') ###
        g.ndata['v_feature'] = inputs
        # g.send(g.edges(),gcn_message)
        # g.recv(g.nodes(),gcn_reduce)
        g.update_all(gcn_message,gcn_reduce)
        v_feature = g.ndata.pop('v_feature')
        return torch.cat([inputs,v_feature],dim = 1) #rel: concate

## SubNetwork

In [25]:
#Subgraph
class SubNetwork(nn.Module):
    def __init__(self,in_feats,hidden_size,layernums):
        super(SubNetwork,self).__init__()
        self.encoder = []
        self.gcnlayer = []
        self.layernums = layernums
        input_size = in_feats
        for i in range(0,layernums):
            if i == 0:
                self.encoder.append(MLP(input_size, hidden_size))
            else:
                self.encoder.append(MLP(hidden_size*2, hidden_size))  #2
            self.gcnlayer.append(GCNLayer(hidden_size, hidden_size*2)) #2

    def forward(self,g,inputs):
        # print('##### subnetwork forward #####')
        g.ndata['v_feature'] = inputs
        g_batch = g
        for i in range(self.layernums):
            g_list = dgl.unbatch(g_batch)
            #进入MLP需要unbatch
            for subg in g_list:
                v_feature = self.encoder[i](subg.ndata['v_feature'])  #node encoder
                subg.ndata['v_feature'] = v_feature
            #进入gcn可以batch回来
            g_batch = dgl.batch(g_list)
            v_feature = self.gcnlayer[i](g_batch,g_batch.ndata['v_feature'])  #gcn
            # print('layer v_feature0: ', v_feature[0])
            g_batch.ndata['v_feature'] = v_feature
        g_list = dgl.unbatch(g_batch)
        v_feature = []
        for subg in g_list:
            v_feature.append(subg.ndata['v_feature'])
        # print('final v_feature0: ', v_feature[0])
        output = torch.stack(v_feature,0)
        # print('subnetwork output ', output.size())
        return output

## attention

In [26]:
class SelfAttention(nn.Module):
    def __init__(self,in_dim,out_dim):
        super(SelfAttention, self).__init__()
        self.wq = torch.nn.Parameter(torch.randn(in_dim,out_dim))
        self.wk = torch.nn.Parameter(torch.randn(in_dim,out_dim))
        self.wv = torch.nn.Parameter(torch.randn(in_dim,out_dim))

    def forward(self,input_feature):
        WQ = input_feature.mm(self.wq)
        WK = input_feature.mm(self.wk)
        WV = input_feature.mm(self.wv)
        QK = WQ.mm(WK.T)
        QK = F.softmax(QK,dim = 1)
        V = QK.mm(WV)
        return V

## GAT
Global attention graph  \\
建模 polyline node features 间高阶交互

In [27]:
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = None #don't define the g before
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        # gain = nn.init.calculate_gain('relu')
        gain = nn.init.calculate_gain('leaky_relu')
        nn.init.xavier_normal_(self.fc.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)  #aggregate
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h }

    def forward(self, g,h):
        self.g = g
        # equation (1) encoder polyline
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2) aggregate
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)  GNN
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

## MTR


In [37]:
#MTR model
class MTR(nn.Module):
    '''
    调用此模型即可
    '''
    def __init__(self,in_dim,hidden_size,out_dim):
        super(MTR,self).__init__()
        self.subMapNetwork = SubNetwork(in_dim,hidden_size,3)
        self.subAgentNetwork = SubNetwork(in_dim,hidden_size,3)
        self.subObjNetwork = SubNetwork(in_dim,hidden_size,3)
        self.GlobalNetwork = SelfAttention(hidden_size*2,hidden_size*2) #GATLayer(hidden_size*2,hidden_size*2)
        self.MLP = [nn.Sequential(
            nn.Linear(hidden_size*2,hidden_size*2),
            #nn.LayerNorm(hidden_size),
            nn.Tanh(), #ReLU?
            nn.Linear(hidden_size*2,out_dim)
        ) for i in range(num_agents)]

    def forward(self,agent_set,map_set,agent_feature,map_feature,map_mask, obj_set,obj_feature):
        MapOutputs = []

        agent_feature = torch.where(torch.isnan(agent_feature),torch.full_like(agent_feature, 0), agent_feature)
        # print('Agent network 0 ', agent_feature[0])

        Globalfeature = torch.max(self.subAgentNetwork(agent_set[0],agent_feature[0]), dim=1)[0].unsqueeze(0)  #agent gnn
        # print('Globalfeature0 ', Globalfeature.size(), Globalfeature)

        Globalfeature = torch.where(torch.isnan(Globalfeature),torch.full_like(Globalfeature, 0), Globalfeature)
        # print('Globalfeature0 fill 0 ', Globalfeature.size(), Globalfeature)

        for i,graph in enumerate(agent_set):  #concate map attn
            if i>0:
                # print('########## subAgentNetwork ##########')
                Globalfeature = torch.cat((Globalfeature,torch.max(self.subAgentNetwork(graph,agent_feature[i]), dim=1)[0].unsqueeze(0)),0)

        max_mask = torch.max(torch.sum(map_mask,dim = 1),dim = 0)[0].int()
        # nodeN = 1 + max_mask
        for i,graph in enumerate(map_set):  #concate map attn
            if i >= max_mask:
                break
            # print('########## subMapNetwork ##########')
            Globalfeature = torch.cat((Globalfeature,torch.max(self.subMapNetwork(graph,map_feature[i]), dim=1)[0].unsqueeze(0)),0)

        #新增obj
        for i,graph in enumerate(obj_set):  #concate map attn
            # if i >= max_mask:
            #     break
            # print('########## subObjNetwork ##########')
            Globalfeature = torch.cat((Globalfeature,torch.max(self.subObjNetwork(graph,obj_feature[i]), dim=1)[0].unsqueeze(0)),0)

        # print('GlobalfeatureObj - 0 \n', Globalfeature[0])
        v_feature = []
        for i in range(Globalfeature.shape[1]):
            # print('########## GlobalNetwork ##########')
            v_feature.append(self.GlobalNetwork(Globalfeature[:,i])[0])
            # print('v_feature ', v_feature[0])
        #输出4辆agents特征
        outputs = [self.MLP[i](torch.stack(v_feature,0)) for i in range(num_agents)]
        print('MTR outputs 0-1: \n', outputs[0], '\n', outputs[1])
        # return self.MLP(torch.stack(v_feature,0))
        return torch.stack(outputs,dim = 0) #4*50

    def save(self,name=None):
        if name is None:
            prefix = '/content/gdrive/MyDrive/exitD/checkpoints/' + 'MTR' + '_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(),name)
        return name

    def load(self,path):
        self.load_state_dict(torch.load(path))

# Train

test

In [29]:

def train(**kwargs):
    '''
    训练
    '''
    opt.parse(kwargs)
    if opt.load_model_path:
        model.load(opt.load_model_path)
    if opt.use_gpu: model.cuda()

    train_data = MTRDataset(opt.train_data_root,train = True)
    val_data = MTRDataset(opt.train_data_root,train = False)
    train_dataloader = DataLoader(train_data,opt.batch_size,shuffle = True,num_workers = opt.num_workers,collate_fn=collate)
    val_dataloader = DataLoader(val_data,opt.batch_size,shuffle = False,num_workers = opt.num_workers,collate_fn=collate)

    criterion = torch.nn.MSELoss()
    lr = opt.lr
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay = opt.weight_decay)
    TrainingLoss = []
    ValLoss = []
    epochs = []
    preloss = 1e100
    for epoch in range(opt.max_epoch):
        print('#################### epoch ', epoch, ' ####################')
        losses = 0
        num = 0
        for ii,(data,label) in enumerate(train_dataloader):
            print('############### ii ', ii, ' ###############')
            print('data agent', len(data['Agent']), '\n', data['Agent'][0])
            #input = Variable(data)
            target = Variable(label)
            if opt.use_gpu:
                data['Agent'] = data['Agent'].to(torch.device('cuda:0'))
                for g in data['Map']:
                    g = g.to(torch.device('cuda:0'))
                data['Agentfeature'] = Variable(data['Agentfeature']).cuda()
                for feature in data['Mapfeature']:
                    feature = Variable(feature).cuda()
                target = target.cuda()
            if len(data['Map']) == 0:
                continue
            optimizer.zero_grad()
            # score = model(data['Agent'],data['Map'],data['Agentfeature'],data['Mapfeature'],data['Mapmask'])
            score = model(data['Agent'],data['Map'],data['Agentfeature'], \
                  data['Mapfeature'],data['Mapmask'], data['Obj'],data['Objfeature'])
            # loss = criterion(score.double().reshape(-1,60),target.double())
            print('score original size ', score.size())
            # score = torch.flatten(score.double().reshape(-1,150), end_dim=1)
            score = score.double().reshape(-1,150)
            print('score - target size', score.size(), target.size())

            loss = criterion(score,target.double())
            print(score[0], '\n', target[0])
            loss.backward()
            optimizer.step()
            losses += loss.data
            num += 1
            if num % 10 == 0:
                print(num)
        model.save()
        epochs.append(epoch)
        TrainingLoss.append(losses/num)
        print('#################### Training: ',losses/num, ' ####################')
        ValLoss.append(val(model,val_dataloader))
        if losses/num > preloss:
            lr = lr*opt.lr_decay

        preloss = losses/num

    plt.figure()
    plt.title('loss during training')  #标题
    plt.plot(epochs, TrainingLoss, label="TrainingLoss")
    plt.plot(epochs, ValLoss, label="ValLoss")
    plt.legend()
    plt.grid()
    plt.show()
    plt.savefig('/content/gdrive/MyDrive/exitD/data/plt/' + 'MTR_' + '%s.jpg' \
                % datetime.datetime.now().strftime("%m%d_%H%M"))

    model.save('new.pth')

@torch.no_grad()
def val(model, dataloader):
    '''
    计算模型在验证集上的准确率等信息，用以辅助训练
    '''
    model.eval()
    losses = 0
    num = 0
    criterion = torch.nn.MSELoss()
    for ii,(data,label) in enumerate(dataloader):
        #input = Variable(data)
        target = Variable(label)
        if opt.use_gpu:
            data['Agent'] = data['Agent'].to(torch.device('cuda:0'))
            for g in data['Map']:
                g = g.to(torch.device('cuda:0'))
            data['Agentfeature'] = data['Agentfeature'].cuda()
            for feature in data['Mapfeature']:
                feature = Variable(feature).cuda()
            target = target.cuda()
        if len(data['Map']) == 0:
            continue
        score = model(data['Agent'],data['Map'],data['Agentfeature'], \
                  data['Mapfeature'],data['Mapmask'],data['Obj'],data['Objfeature'])
        loss = criterion(score.double().reshape(-1,150),target.double())
        losses += loss.data
        num += 1
        if num % 10 == 0:
            print(num)
    model.train()
    print('#################### Eval:',losses/num, ' ####################')
    return losses/num

### test

In [ ]:
# df = pd.read_csv('/content/gdrive/MyDrive/exitD/data/train_mtr/3.csv')
# df.head()

,index,frame,trackId,laneletId,xCenter,yCenter,xVelocity,yVelocity,width,length,heading,leadId,rearId,leftLeadId,rightLeadId,leftRearId,rightRearId,recordingId,class
0,1413,200,10,1676,576.1485,-473.7247,18.2565,-24.1354,1.852,4.1065,307.1046,-1,15,-1,-1,11,-1,0,car
1,1414,201,10,1676,576.8784,-474.6901,18.2463,-24.1347,1.852,4.1065,307.0900,-1,15,-1,-1,11,-1,0,car
2,1415,202,10,1676,577.6080,-475.6556,18.2361,-24.1339,1.852,4.1065,307.0755,-1,15,-1,-1,11,-1,0,car
3,1416,203,10,1676,578.3372,-476.6211,18.2260,-24.1330,1.852,4.1065,307.0612,-1,15,-1,-1,11,-1,0,car
4,1417,204,10,1676,579.0660,-477.5866,18.2158,-24.1321,1.852,4.1065,307.0469,-1,15,-1,-1,11,-1,0,car


In [30]:
train_data = MTRDataset(opt.train_data_root,train = True)
train_data[0]

/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


({'centerAgent': [410.4314, -232.6439, -18.042, 23.3417, 127.7022],
  'Map': [Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=6,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feature': Scheme(shape=(11,), dtype=torch.float32)}
         edata_schemes={}),
   Graph(num_nodes=20, num_edges=0,
         ndata_schemes={'v_feat

In [ ]:
train_dataloader = DataLoader(train_data,opt.batch_size,shuffle = True,num_workers = opt.num_workers,collate_fn=collate)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# main

In [31]:
opt = DefaultConfig()
model = MTR(11,64,150)  #30s * 5

In [ ]:
train()

Output hidden; open in https://colab.research.google.com to view.

torch.Size([4, 50, 150]) torch.Size([200, 150])
torch.Size([4, 50, 150]) torch.Size([200, 150])
torch.Size([4, 40, 150]) torch.Size([160, 150])
#################### Training:  tensor(nan, dtype=torch.float64)  ####################

In [40]:
class DefaultConfig(object):
    env = 'default' # visdom 环境
    model = 'MTR' # 使用的模型，名字必须与models/__init__.py中的名字一致

    train_data_root = '/content/gdrive/MyDrive/exitD/data/train_mtr' # 训练集存放路径
    # test_data_root = './data/test' # 测试集存放路径
    load_model_path = None # 加载预训练的模型的路径，为None代表不加载

    batch_size = 20 # batch size
    use_gpu = False # use GPU or not
    num_workers = 4 # how many workers for loading data
    print_freq = 20 # print info every N batch

    # agent
    num_agents = 4

    # map feature
    num_points_polyline = 10
    num_src_polyline = 20

    result_file = 'result.csv'

    max_epoch = 100  #test epoch
    lr = 0.01 # initial learning rate
    lr_decay = 0.97 # when val_loss increase, lr = lr*lr_decay
    weight_decay = 1e-4 # 损失函数

    def parse(self, kwargs):
        '''
        根据字典kwargs 更新 config参数
        '''
        # 更新配置参数
        for k, v in kwargs.items():
            if not hasattr(self, k):
                # 警告还是报错，取决于你个人的喜好
                warnings.warn("Warning: opt has not attribut %s" %k)
            setattr(self, k, v)

        # 打印配置信息
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('__'):
                print(k, getattr(self, k))

### test

In [ ]:
# t = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]]) #2,2,2
# torch.flatten(t, end_dim=1) #4,2

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

In [41]:
opt = DefaultConfig()
model = MTR(11,64,150)  #30s * 5

In [42]:
train()

Output hidden; open in https://colab.research.google.com to view.

Globalfeature0
 tensor([[[0.0000, 1.1397, 0.5596,  ..., 0.0000, 0.2401, 1.4283],
         [0.0000, 1.2795, 1.2820,  ..., 0.0000, 1.6376, 1.4272],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         ...,
         [0.4314, 1.3321, 0.5596,  ..., 0.0000, 0.1900, 0.0000],
         [0.0000, 0.6568, 0.7173,  ..., 0.0808, 1.0348, 0.0000],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan]]],
       grad_fn=<UnsqueezeBackward0>)

GlobalfeatureObj - 0
 tensor([[0.0000, 1.1397, 0.5596,  ..., 0.0000, 0.2401, 1.4283],
        [0.0000, 1.2795, 1.2820,  ..., 0.0000, 1.6376, 1.4272],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        ...,
        [0.4314, 1.3321, 0.5596,  ..., 0.0000, 0.1900, 0.0000],
        [0.0000, 0.6568, 0.7173,  ..., 0.0808, 1.0348, 0.0000],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan]],
       grad_fn=<SelectBackward0>)
MTR outputs:
 tensor([[-0.4604, -0.4431, -0.2058,  ...,  0.4031,  0.6091,  0.1878],
        [-0.3673, -0.6518, -0.1386,  ...,  0.5441,  0.3441,  0.2402],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        ...,
        [ 0.4526, -0.3266,  1.4612,  ...,  0.6087,  0.4114, -0.2454],
        [ 0.0453, -1.1737,  0.8518,  ...,  0.2482,  0.3167, -0.6066],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],
       grad_fn=<AddmmBackward0>)
torch.Size([4, 10, 150]) torch.Size([40, 150])
tensor([[-0.4604, -0.4431, -0.2058,  ...,  0.4031,  0.6091,  0.1878],
        [-0.3673, -0.6518, -0.1386,  ...,  0.5441,  0.3441,  0.2402],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        ...,
        [ 0.4526, -0.3266,  1.4612,  ...,  0.6087,  0.4114, -0.2454],
        [ 0.0453, -1.1737,  0.8518,  ...,  0.2482,  0.3167, -0.6066],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],